In [19]:
import dask.array as da
import numpy as np
import h5py as h5
from tqdm import tqdm
from pandas import DataFrame

from sklearn.externals import joblib

In [46]:
DATA_PATH = 'hackathon_data_pred_students_pred_from_full_pred_eighties_from_full_1_without_gt.h5'
CHUNK_SIZE = 1000
MODEL_PATH = 'vc_20trees_balanced.pkl'
CSV_OUTPUT = 'vc_20trees_balanced.csv'

In [29]:
# Custom VotingClassifier
# Can ensemble pre-fitted models (as opposed to sklearn's implementation which needs retraining)

class VotingClassifier:
    # Two voting modes :
        # - Hard: the majority wins, if there's a tie, a contender is selected randomly
        # - Soft: use the provided weights and tally the weighted votes to determine winners. Weights must have shape (len(estimators))
    
    def __init__(self, estimators, classes=None, weights=None):
        # estimators: iterable of estimators implementing the predict() API
        # weights: iterable of (numeric) weights
        # classes: list of matching classes
        
        self.estimators = estimators
        self.weights = np.array(weights)
        self.n_classes = len(self.estimators[0].n_classes_)
        
    
    def _special_argmax(self, pred):
        # Compute the usual argmax along the lines
        # For each null predictions ([0]*no_features), replace the argmax with -1 to filter it easily
        # Having an argmax of 0 then actually means that the predicted label is the #0

        # pred shape=(no_classifiers, no_features)
        out = pred.argmax(axis=1)
        out[np.all(pred==-1.0, axis=1)] = self.n_classes + 1

        return out

    def predict(self, X):
        # X is of shape (no_samples, no_features)
        # Return array of shape (no_samples,)

        predictions = np.empty((len(self.estimators), X.shape[0], self.n_classes))
        
        for i, clf in enumerate(self.estimators):
            predictions[i, :, :] = clf.predict(X)
            
        # predictions is of shape (no_classifiers, no_samples, no_features)
        
        out = np.zeros((X.shape[0],)) # out matrix
        
        for k in range(predictions.shape[1]):
            arr = predictions[:, k, :]
            
            # Replace all the 0 to -1 (in order to filter out the null prediction later on)
            np.place(arr, arr==0.0, -1)
            
            arr_label = self._special_argmax(arr)
            
            idx_label = ~(arr_label == self.n_classes + 1)
            
            arr_label = arr_label[idx_label]

            if len(arr_label) == 0:
                # If an image cannot be predicted at all by any estimator, we simply affect it a random class unfortunately
                choice = np.random.choice(range(self.n_classes))
                print('Unpredictable image at index {}, affected class -> {}'.format(k, choice))
                out[k] = choice
                continue
            
            out[k] = np.argmax(np.bincount(arr_label, 
                                           weights=self.weights[idx_label]))
            
        return out    
    
    def score(self, X, y):
        # Convert y to its category label
        y = y.argmax(axis=1)
        y_pred = self.predict(X)
        
        accuracy = accuracy_score(y, y_pred)
        fbeta = fbeta_score(y, y_pred, 2, average='macro')
        
        print('Accuracy :', accuracy)
        print('F2 score :', fbeta)

In [31]:
# Load model
vc = joblib.load(MODEL_PATH)

C:\Program Files\Miniconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Program Files\Miniconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
# Setup generator yielding X, y chunks (default size 1000)

def generator(h5_path, chunk_size, train_size):
    f = h5.File(h5_path, 'r')
    X = f['S2']
    
    # Compute number of chunks needed
    no_chunks = len(X) // chunk_size
    if len(X) % chunk_size > 0:
        # len(X) is not an exact multiple of chunk_size
        no_chunks += 1
    
    print(len(X), '-', no_chunks)
    for c in range(no_chunks):
        X_to_yield = X[c*chunk_size:(c+1)*chunk_size,:,:,:]    
        
        # Wrangle data
        X_to_yield = np.array(X_to_yield).reshape((-1, 16*16*4)) # Flatten each element in the array
        
        yield X_to_yield
# Test
# for X, y in generator('data/eightieth.h5', 1000):
#     pass

In [32]:
predictions = np.empty(len(h5.File(DATA_PATH)['S2']))

for c, X in enumerate(generator(DATA_PATH, CHUNK_SIZE, 1/8)):
    predictions[c*CHUNK_SIZE:(c+1)*CHUNK_SIZE] = vc.predict(X)

241700 - 242
Unpredictable image at index 215, affected class -> 6
Unpredictable image at index 216, affected class -> 11
Unpredictable image at index 217, affected class -> 11
Unpredictable image at index 218, affected class -> 0
Unpredictable image at index 219, affected class -> 1
Unpredictable image at index 224, affected class -> 9
Unpredictable image at index 233, affected class -> 5
Unpredictable image at index 235, affected class -> 1
Unpredictable image at index 236, affected class -> 0
Unpredictable image at index 248, affected class -> 3
Unpredictable image at index 253, affected class -> 2
Unpredictable image at index 259, affected class -> 0
Unpredictable image at index 261, affected class -> 1
Unpredictable image at index 268, affected class -> 7
Unpredictable image at index 271, affected class -> 7
Unpredictable image at index 290, affected class -> 1
Unpredictable image at index 291, affected class -> 7
Unpredictable image at index 306, affected class -> 4
Unpredictable

Unpredictable image at index 935, affected class -> 2
Unpredictable image at index 940, affected class -> 9
Unpredictable image at index 941, affected class -> 13
Unpredictable image at index 942, affected class -> 6
Unpredictable image at index 948, affected class -> 7
Unpredictable image at index 951, affected class -> 9
Unpredictable image at index 954, affected class -> 5
Unpredictable image at index 964, affected class -> 5
Unpredictable image at index 972, affected class -> 4
Unpredictable image at index 994, affected class -> 11
Unpredictable image at index 999, affected class -> 6
Unpredictable image at index 50, affected class -> 3
Unpredictable image at index 108, affected class -> 0
Unpredictable image at index 111, affected class -> 10
Unpredictable image at index 113, affected class -> 6
Unpredictable image at index 121, affected class -> 8
Unpredictable image at index 130, affected class -> 6
Unpredictable image at index 173, affected class -> 9
Unpredictable image at ind

Unpredictable image at index 839, affected class -> 2
Unpredictable image at index 841, affected class -> 0
Unpredictable image at index 842, affected class -> 13
Unpredictable image at index 843, affected class -> 8
Unpredictable image at index 844, affected class -> 6
Unpredictable image at index 845, affected class -> 12
Unpredictable image at index 846, affected class -> 10
Unpredictable image at index 847, affected class -> 0
Unpredictable image at index 848, affected class -> 5
Unpredictable image at index 849, affected class -> 10
Unpredictable image at index 850, affected class -> 0
Unpredictable image at index 874, affected class -> 13
Unpredictable image at index 878, affected class -> 5
Unpredictable image at index 13, affected class -> 4
Unpredictable image at index 19, affected class -> 7
Unpredictable image at index 69, affected class -> 0
Unpredictable image at index 102, affected class -> 1
Unpredictable image at index 103, affected class -> 8
Unpredictable image at ind

Unpredictable image at index 565, affected class -> 4
Unpredictable image at index 566, affected class -> 7
Unpredictable image at index 567, affected class -> 0
Unpredictable image at index 612, affected class -> 6
Unpredictable image at index 616, affected class -> 0
Unpredictable image at index 627, affected class -> 5
Unpredictable image at index 628, affected class -> 12
Unpredictable image at index 635, affected class -> 10
Unpredictable image at index 639, affected class -> 9
Unpredictable image at index 640, affected class -> 11
Unpredictable image at index 675, affected class -> 12
Unpredictable image at index 678, affected class -> 0
Unpredictable image at index 679, affected class -> 7
Unpredictable image at index 680, affected class -> 11
Unpredictable image at index 694, affected class -> 0
Unpredictable image at index 696, affected class -> 12
Unpredictable image at index 702, affected class -> 0
Unpredictable image at index 703, affected class -> 2
Unpredictable image at

Unpredictable image at index 715, affected class -> 10
Unpredictable image at index 716, affected class -> 1
Unpredictable image at index 720, affected class -> 6
Unpredictable image at index 723, affected class -> 8
Unpredictable image at index 726, affected class -> 10
Unpredictable image at index 727, affected class -> 5
Unpredictable image at index 728, affected class -> 12
Unpredictable image at index 729, affected class -> 9
Unpredictable image at index 730, affected class -> 10
Unpredictable image at index 747, affected class -> 5
Unpredictable image at index 767, affected class -> 12
Unpredictable image at index 771, affected class -> 10
Unpredictable image at index 774, affected class -> 3
Unpredictable image at index 779, affected class -> 6
Unpredictable image at index 784, affected class -> 12
Unpredictable image at index 791, affected class -> 6
Unpredictable image at index 792, affected class -> 11
Unpredictable image at index 795, affected class -> 7
Unpredictable image 

...

Unpredictable image at index 17, affected class -> 4
Unpredictable image at index 49, affected class -> 7
Unpredictable image at index 68, affected class -> 10
Unpredictable image at index 69, affected class -> 7
Unpredictable image at index 71, affected class -> 11
Unpredictable image at index 72, affected class -> 10
Unpredictable image at index 80, affected class -> 9
Unpredictable image at index 81, affected class -> 10
Unpredictable image at index 86, affected class -> 5
Unpredictable image at index 87, affected class -> 10
Unpredictable image at index 91, affected class -> 5
Unpredictable image at index 101, affected class -> 3
Unpredictable image at index 102, affected class -> 0
Unpredictable image at index 104, affected class -> 5
Unpredictable image at index 120, affected class -> 10
Unpredictable image at index 121, affected class -> 6
Unpredictable image at index 126, affected class -> 13
Unpredictable image at index 136, affected class -> 2
Unpredictable image at index 145

Unpredictable image at index 311, affected class -> 13
Unpredictable image at index 314, affected class -> 1
Unpredictable image at index 317, affected class -> 4
Unpredictable image at index 324, affected class -> 11
Unpredictable image at index 325, affected class -> 13
Unpredictable image at index 326, affected class -> 12
Unpredictable image at index 328, affected class -> 1
Unpredictable image at index 329, affected class -> 0
Unpredictable image at index 334, affected class -> 0
Unpredictable image at index 342, affected class -> 9
Unpredictable image at index 343, affected class -> 3
Unpredictable image at index 349, affected class -> 0
Unpredictable image at index 350, affected class -> 3
Unpredictable image at index 351, affected class -> 5
Unpredictable image at index 352, affected class -> 12
Unpredictable image at index 354, affected class -> 5
Unpredictable image at index 358, affected class -> 1
Unpredictable image at index 359, affected class -> 8
Unpredictable image at 

Unpredictable image at index 893, affected class -> 2
Unpredictable image at index 903, affected class -> 8
Unpredictable image at index 919, affected class -> 5
Unpredictable image at index 924, affected class -> 6
Unpredictable image at index 941, affected class -> 7
Unpredictable image at index 955, affected class -> 2
Unpredictable image at index 957, affected class -> 1
Unpredictable image at index 961, affected class -> 9
Unpredictable image at index 13, affected class -> 7
Unpredictable image at index 18, affected class -> 13
Unpredictable image at index 35, affected class -> 9
Unpredictable image at index 37, affected class -> 12
Unpredictable image at index 38, affected class -> 7
Unpredictable image at index 39, affected class -> 4
Unpredictable image at index 40, affected class -> 10
Unpredictable image at index 49, affected class -> 10
Unpredictable image at index 55, affected class -> 7
Unpredictable image at index 77, affected class -> 3
Unpredictable image at index 78, a

Unpredictable image at index 940, affected class -> 4
Unpredictable image at index 941, affected class -> 6
Unpredictable image at index 942, affected class -> 12
Unpredictable image at index 943, affected class -> 6
Unpredictable image at index 951, affected class -> 6
Unpredictable image at index 955, affected class -> 9
Unpredictable image at index 957, affected class -> 13
Unpredictable image at index 958, affected class -> 7
Unpredictable image at index 959, affected class -> 4
Unpredictable image at index 960, affected class -> 10
Unpredictable image at index 961, affected class -> 10
Unpredictable image at index 963, affected class -> 1
Unpredictable image at index 964, affected class -> 12
Unpredictable image at index 966, affected class -> 1
Unpredictable image at index 968, affected class -> 9
Unpredictable image at index 969, affected class -> 8
Unpredictable image at index 972, affected class -> 5
Unpredictable image at index 973, affected class -> 2
Unpredictable image at 

Unpredictable image at index 558, affected class -> 3
Unpredictable image at index 563, affected class -> 7
Unpredictable image at index 564, affected class -> 1
Unpredictable image at index 565, affected class -> 10
Unpredictable image at index 576, affected class -> 7
Unpredictable image at index 579, affected class -> 5
Unpredictable image at index 581, affected class -> 2
Unpredictable image at index 582, affected class -> 12
Unpredictable image at index 585, affected class -> 11
Unpredictable image at index 596, affected class -> 3
Unpredictable image at index 597, affected class -> 12
Unpredictable image at index 600, affected class -> 8
Unpredictable image at index 601, affected class -> 5
Unpredictable image at index 609, affected class -> 4
Unpredictable image at index 610, affected class -> 12
Unpredictable image at index 648, affected class -> 6
Unpredictable image at index 664, affected class -> 7
Unpredictable image at index 665, affected class -> 3
Unpredictable image at 

In [37]:
predictions

array([ 9.,  9.,  9., ..., 12., 12.,  5.])

In [48]:
predictions_df = DataFrame(predictions.astype(int), columns=['TOP_LANDCOVER'], index=None)
predictions_df.head()

,TOP_LANDCOVER
0,9
1,9
2,9
3,9
4,9


In [49]:
predictions_df.to_csv(CSV_OUTPUT, index_label='ID')

In [ ]:
pred_idx = get_idxs(PATH_PREDICT_WITHOUT_GT)
print(len(pred_idx))
pred_gen = prediction_generator(PATH_PREDICT_WITHOUT_GT, BATCH_SIZE, pred_idx)
prediction = model.predict_generator(pred_gen, steps=get_batch_count(pred_idx, BATCH_SIZE), verbose=1)
print(len(prediction))
# build_h5_pred_file(np.argmax(prediction, axis = 1), PATH_SUBMIT)
panda_prediction = pd.DataFrame(np.argmax(prediction, axis = 1))
panda_prediction.columns = ['TOP_LANDCOVER']
panda_prediction.head()
# panda_prediction.describe()
panda_prediction.to_csv(PATH_SUBMIT, index_label = 'ID')